## 2. Chargement et Préparation des Données

### 2.1 Importation des données

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import shap

In [12]:
import pandas as pd

# =========================
# 1) Load and Merge
# =========================

# Load CSVs
employee_df = pd.read_csv("data/employee_survey_data.csv")
general_df = pd.read_csv("data/general_data.csv")
manager_df = pd.read_csv("data/manager_survey_data.csv")
in_time_df = pd.read_csv("data/in_time.csv")
out_time_df = pd.read_csv("data/out_time.csv")

# Merge employee_df and general_df on EmployeeID
merged_df = pd.merge(employee_df, general_df, on="EmployeeID", how="inner")

# Merge in manager_df
merged_df = pd.merge(merged_df, manager_df, on="EmployeeID", how="inner")

# =========================
# 2) Drop Irrelevant Columns
# =========================

# Example columns you might remove:
#   - Over18: Single value ‘Y’ for everyone
#   - EmployeeCount: Single value
#   - StandardHours: Single value (often 8)
# Adjust to your data as needed
columns_to_drop = ["EmployeeCount", "Gender", "MaritalStatus"]
merged_df.drop(columns=columns_to_drop, axis=1, inplace=True, errors="ignore")

# =========================
# 3) Clean Up and Prepare Time Data
# =========================

# Rename 'Unnamed: 0' to 'EmployeeID' for easier merging
in_time_df.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
out_time_df.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)

# We’ll create a separate DataFrame that holds each employee’s average work hours
# Convert time columns to datetime (ignore errors if any NaNs or incorrect formats appear)
time_cols = in_time_df.columns[1:]  # skip EmployeeID column
in_time_df[time_cols] = in_time_df[time_cols].apply(pd.to_datetime, errors="coerce")
out_time_df[time_cols] = out_time_df[time_cols].apply(pd.to_datetime, errors="coerce")

# Calculate the daily hours as OUT - IN
# This returns a DataFrame of timedeltas; convert to hours by dividing total_seconds by 3600
daily_hours = out_time_df[time_cols].sub(in_time_df[time_cols])
daily_hours = daily_hours.apply(lambda row: row.dt.total_seconds() / 3600)

# Create average hours feature per employee
avg_hours_df = pd.DataFrame({
    "EmployeeID": in_time_df["EmployeeID"],
    "AverageWorkHours": daily_hours.mean(axis=1)  # mean across all days
})

# =========================
# 4) Merge Time Features Back
# =========================

final_df = pd.merge(merged_df, avg_hours_df, on="EmployeeID", how="left")

# =========================
# 5) Create or Transform Other Meaningful Features
# =========================

# Example: Combine environment satisfaction & job satisfaction into a single “OverallSatisfaction”
final_df["OverallSatisfaction"] = final_df[["EnvironmentSatisfaction", "JobSatisfaction"]].mean(axis=1)

# Example: Convert ‘DistanceFromHome’ from numeric to a bucket feature
# (e.g., 0–5 = “Close”, 6–10 = “Medium”, etc.)
def distance_bucket(dist):
    if dist <= 5:
        return "Close"
    elif dist <= 10:
        return "Medium"
    else:
        return "Far"

final_df["DistanceBucket"] = final_df["DistanceFromHome"].apply(distance_bucket)

# Done! You can now use final_df for further analysis or modeling.
final_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating,AverageWorkHours,OverallSatisfaction,DistanceBucket
0,1,3.0,4.0,2.0,51,No,Travel_Rarely,Sales,6,2,...,1.0,6,1,0,0,3,3,7.373651,3.5,Medium
1,2,3.0,2.0,4.0,31,Yes,Travel_Frequently,Research & Development,10,1,...,6.0,3,5,1,4,2,4,7.718969,2.5,Medium
2,3,2.0,2.0,1.0,32,No,Travel_Frequently,Research & Development,17,4,...,5.0,2,5,0,3,3,3,7.013240,2.0,Far
3,4,4.0,4.0,3.0,38,No,Non-Travel,Research & Development,2,5,...,13.0,5,8,7,5,2,3,7.193678,4.0,Close
4,5,4.0,1.0,3.0,32,No,Travel_Rarely,Research & Development,10,1,...,9.0,2,6,0,4,3,3,8.006175,2.5,Medium


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
 4   Age                      4410 non-null   int64  
 5   Attrition                4410 non-null   object 
 6   BusinessTravel           4410 non-null   object 
 7   Department               4410 non-null   object 
 8   DistanceFromHome         4410 non-null   int64  
 9   Education                4410 non-null   int64  
 10  EducationField           4410 non-null   object 
 11  JobLevel                 4410 non-null   int64  
 12  JobRole                  4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [15]:
final_df.head()
df = final_df

## Préparation des Données
### 1. Gestion des valeurs manquantes

In [ ]:
# Vérifier les valeurs manquantes
df.isnull().sum()

# Imputer les valeurs manquantes pour les variables numériques avec la médiane
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

# Sélection des colonnes numériques

## à Compléter
df_num = df.select_dtypes(include=float)
imputer.fit(df_num)
## à Compléter

# Remplacement des valeurs manquantes
df_num_imputed = pd.DataFrame(imputer.transform(df_num), columns=df_num.columns)

# Vérifier s'il reste des valeurs manquantes
df_num_imputed.isnull().sum()

EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
NumCompaniesWorked         0
TotalWorkingYears          0
AverageWorkHours           0
OverallSatisfaction        0
dtype: int64

### 2. Encodage des variables catégorielles

In [11]:
# Sélection des colonnes catégorielles
df_cat = df.select_dtypes(include=[object])

# Encodage one-hot des variables catégorielles

## à Compléter
df_cat_encoded = pd.get_dummies(df_cat)
## à Compléter 

# Vérification du résultat de l'encodage
df_cat_encoded.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,DistanceBucket_Close,DistanceBucket_Far,DistanceBucket_Medium
0,True,False,False,False,True,False,False,True,False,True,...,False,False,False,False,True,False,True,False,False,True
1,False,True,False,True,False,False,True,False,False,True,...,True,False,False,False,False,True,True,False,False,True
2,True,False,False,True,False,False,True,False,False,False,...,False,True,False,False,True,False,True,False,True,False
3,True,False,True,False,False,False,True,False,False,True,...,False,False,False,False,True,False,True,True,False,False
4,True,False,False,False,True,False,True,False,False,False,...,False,True,False,False,False,True,True,False,False,True


### 3. Normalization des données

### 4. Assemblage des données préparées

### 5. Séparation des caractéristiques (X) et de la cible (y)

## Exploration et Visualisation des Données

### 1. Analyse des Statistiques Descriptives

Avant de plonger dans la visualisation des données, examinons les statistiques descriptives pour obtenir une première idée des distributions des variables.

### 2. Tests d'ANOVA & X2
write something

### 3. Visualisation des Variables Numériques
Nous commencerons par visualiser les distributions des variables numériques pour mieux comprendre leurs comportements.

### 3. Matrice de Corrélation
Pour explorer les relations entre les variables numériques, nous utilisons une matrice de corrélation et une heatmap.

### 4. Visualisation des Relations entre les Variables
Nous allons créer quelques graphes de dispersion pour visualiser les relations entre certaines variables clés.

### 5. Analyse de la Variable Cible
Analysons la variable cible Sold6M pour voir comment elle est distribuée.

### 6. Relations entre les Variables et la Cible
Examinons comment certaines variables influencent la probabilité de vendre un bien immobilier dans les 6 mois.

## Modèles de Classification
Dans cette section, nous allons créer et évaluer plusieurs modèles de classification afin de prédire... (ici on met des definitions rapide et insights sur les modeles (interpretabilité vs performance...))

## Application des modèles
### 0. Préparation des données d'apprentissage et de test

### 1. Régerssion Logistique

Ici, ajouter plusieurs cellules (entrainement, evaluation, optimisation, explicabilité)

### 2. Random Forest

Ici, ajouter plusieurs cellules (entrainement, evaluation, optimisation, explicabilité)

# Etude comparative entre les modèles

## Interprétation des Résultats
### 1. Analyse des Performances

## Conclusion
### 1. Synthèse des Travaux Réalisés